In [7]:
# Data Processing from base data to dataset for residential exemption analysis

In [1]:
import pandas as pd

df_clean = pd.read_json("../../public/data/base_data.json")

years = df_clean['Tax Year'].unique().tolist()
entities = df_clean['Entity Name'].unique().tolist()


In [ ]:
property_types = ['Primary Residential', 'Non-primary Residential', 'Commercial', 'Agricultural', 'Unimproved', 'Total Real Property', 
                  'Personal Property', 
                  'Centrally Assessed']

entities = entities = df_clean["Entity Name"].unique().tolist()
years = df_clean["Tax Year"].unique().tolist()


# Compute primary residential market value
df_clean['Market Value'] = df_clean["Primary Residential"]/.55


# Compute new primary residential exemption (55%)
df_clean['Counterfactual Primary Residential'] = df_clean["Market Value"] * .45


# Compute counterfactual total taxable property using counterfactual primary residential taxable property
df_clean['Counterfactual Total Taxable Property'] = df_clean["Counterfactual Primary Residential"] + df_clean["Non-primary Residential"] + df_clean["Commercial"] + df_clean["Agricultural"] + df_clean["Unimproved"] + df_clean["Personal Property"] + df_clean["Centrally Assessed"]


# Compute counterfactual tax rate using counterfactural total taxable property & existing total tax revenue
df_clean['Counterfactual Tax Rate'] = (df_clean["Revenue, Total"]/ df_clean['Counterfactual Total Taxable Property'])/10


# Compute counterfactual tax revenue by property type using counterfactual tax rate
for type in property_types:
    if type == 'Primary Residential':   
        df_clean['Counterfactual Revenue, ' + type] = df_clean['Counterfactual Tax Rate'] * df_clean['Counterfactual '+ type] * 10

    elif type == "Personal Property":
        for entity in entities:
            for year in years:
                df_clean.loc[(df_clean["Tax Year"]==year) & (df_clean["Entity Name"]==entity), 'Counterfactual Revenue, Personal Property'] = df_clean.loc[(df_clean["Tax Year"]==year-1) & (df_clean["Entity Name"]==entity)]["Counterfactual Tax Rate"].sum() * df_clean.loc[(df_clean["Tax Year"]==year) & (df_clean["Entity Name"]==entity)]["Personal Property"].sum() * 10
    
    else:
        df_clean['Counterfactual Revenue, ' + type] = df_clean['Counterfactual Tax Rate'] * df_clean[type] * 10 


# Compute counterfactual property type shares using counterfactual (primary residential property, taxable value) & existing (non-primary taxable property values) variables
for type in property_types:
        if type == 'Primary Residential':   
            df_clean['Counterfactual ' + type + ', Share'] = df_clean['Counterfactual Primary Residential']/ df_clean['Counterfactual Total Taxable Property']
        else:
            df_clean['Counterfactual ' + type + ', Share'] = df_clean[type]/ df_clean['Counterfactual Total Taxable Property']


# Compute hypothetical liability
df_clean["Primary Residenital Tax Liability"] = 500000 * .55 * df_clean['Tax Rate'] / 100
df_clean["Other Tax Liability"] = 500000 * df_clean['Tax Rate'] / 100
    
df_clean["Counterfactual Primary Residenital Tax Liability"] = 500000 * .45 * df_clean['Counterfactual Tax Rate'] / 100
df_clean["Counterfactual Other Tax Liability"] = 500000 * df_clean['Counterfactual Tax Rate'] / 100


In [11]:
# Export to JSON
df_clean.to_json('../../public/data/exemption_data.json', orient = 'records')
